# PDF AI
The objective of this is to test how to embedd data from pdf

In [ ]:
import os
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import chromadb
os.environ["LANGSMITH_PROJECT"] = "data-chunking-pdf"
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

data = "user-manual"
collection_name = "user-manual"

vector_store = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings,
    client=chroma_client
)
file_path = (
    f'../data/{data}'
)
files_pdf = []
for file in os.listdir(file_path):
    if file.endswith(".pdf"):
        files_pdf.append(os.path.join(file_path, file))

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
for file in files_pdf:
    loader = PyPDFLoader(file)
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunked_documents = text_splitter.split_documents(document)
    vector_store.add_documents(chunked_documents)

    print(f"Added {len(chunked_documents)} chunks to chroma db")

In [ ]:
vector_store.similarity_search("rag")